HeritErrBarsShortcut
E Flynn
5/22/2018

The goal of this notebook is to extract the error bars from the vectors for each trait

In [1]:
source('model_utils.R')
source('heritability_utils.R')
DATA.FOLDER <- "/scratch/PI/mrivas/users/erflynn/sex_div_gwas/data/"

Loading required package: MASS
Loading required package: Matrix
Loading required package: mnormt
Loading required package: qqman

For example usage please run: vignette('qqman')

Citation appreciated but not required:
Turner, S.D. qqman: an R package for visualizing GWAS results using Q-Q and manhattan plots. biorXiv DOI: 10.1101/005165 (2014).

Loading required package: rstan
Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.17.3, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)


In [2]:
trait <- 'whr'
load(file=sprintf("%s/dat_set/dat_%s.RData", DATA.FOLDER, trait))
load(file=sprintf("%s/dat_set/f_%s.RData", DATA.FOLDER, trait))

In [3]:
list_of_draws <- extract(fit1)
pi.draws <- list_of_draws$pi
p <- pi.draws
s.draws <- list_of_draws$Sigma
Sigma <- s.draws

In [4]:
ordered.p <- p[order(p[,2]),] # ordering p by the first component 
head(ordered.p)
p.lower <- ordered.p[0.125*nrow(ordered.p),]
p.upper <- ordered.p[0.975*nrow(ordered.p),]

# how to order the Sigma matrix??
ordered.S <- Sigma[order(Sigma[,1,1]),,]
head(Sigma)
head(ordered.S)


0.7262436,0.2737564
0.7236995,0.2763005
0.7233053,0.2766947
0.7230723,0.2769277
0.7220776,0.2779224
0.7202496,0.2797504


[1] 4.056528e-05 4.081677e-05 3.877695e-05 4.148459e-05 3.939156e-05
[6] 4.174710e-05

[1] 3.774886e-05 3.783435e-05 3.791108e-05 3.794988e-05 3.797542e-05
[6] 3.803775e-05

In [5]:
dat <- labelCategories(dat, getSigma(fit1), getPi(fit1))
table(dat$categories)


     1      2 
275903   5887 

In [6]:
overallHeritability(dat, getSigma(fit1), getPi(fit1))

$`1`
[1] 0.2497339

$`2`
[1] 0.1803766

In [7]:
p.upper # the upper one is higher in the second component
p.lower

[1] 0.6922583 0.3077417

[1] 0.7113659 0.2886341

In [21]:
herit.vals.u <- sapply(1:1600, function(x) overallHeritability(dat, ordered.S[x,,], p.upper))
herit.vals.l <- sapply(1:1600, function(x) overallHeritability(dat, ordered.S[x,,], p.lower))
head(herit.vals.u)
head(herit.vals.l)

In [8]:
# extract the appropriate parts of the S matrix
s.upper <- ordered.S[0.975*1600,,]
s.lower <- ordered.S[0.125*1600,,]

# calculate the heritability
overallHeritability(dat, s.upper, p.upper)
overallHeritability(dat, s.lower, p.lower)

$`1`
[1] 0.2658602

$`2`
[1] 0.1871736

$`1`
[1] 0.2400593

$`2`
[1] 0.1776897

In [9]:
# todo - are the categories calculated at this time or earlier?
# -- turns out earlier --> need to double check
# the results don't change that much tho!

dat2 <- dat
dat2$categories <- NULL
dat.u <- labelCategories(dat2, s.upper, p.upper)
dat.l <- labelCategories(dat2, s.lower, p.lower)


In [10]:
overallHeritability(dat.u, s.upper, p.upper)
overallHeritability(dat.l, s.lower, p.lower)

$`1`
[1] 0.2608714

$`2`
[1] 0.1833282

$`1`
[1] 0.2418546

$`2`
[1] 0.1791825

In [14]:
calcErrBarsHerit <- function(trait){
    load(file=sprintf("%s/dat_set/dat_%s.RData", DATA.FOLDER, trait))
    load(file=sprintf("%s/dat_set/f_%s.RData", DATA.FOLDER, trait))
    
    # extract all estimate
    list_of_draws <- extract(fit1)
    pi.draws <- list_of_draws$pi
    p <- pi.draws
    s.draws <- list_of_draws$Sigma
    Sigma <- s.draws
    
    # extract lower + upper pi
    ordered.p <- p[order(p[,2]),] # ordering p by the non-null component 
    p.lower <- ordered.p[0.125*nrow(ordered.p),]
    p.upper <- ordered.p[0.975*nrow(ordered.p),]

    # extract lower + upper sigma
    ordered.S <- Sigma[order(Sigma[,1,1]),,]
    s.upper <- ordered.S[0.975*1600,,]
    s.lower <- ordered.S[0.125*1600,,]
    
    # recalculate SNP membership
    h <- overallHeritability(dat, getSigma(fit1), getPi(fit1))

    dat2 <- dat
    dat2$categories <- NULL
    dat.u <- labelCategories(dat2, s.upper, p.upper)
    dat.l <- labelCategories(dat2, s.lower, p.lower)

    h.up <- overallHeritability(dat.u, s.upper, p.upper)
    h.low <- overallHeritability(dat.l, s.lower, p.lower)
    return(list("val"=h, "up"=h.up, "low"=h.low))
}

In [15]:
calcErrBarsHerit("whr")


$val
$val$`1`
[1] NaN

$val$`2`
[1] NaN


$up
$up$`1`
[1] 0.2608714

$up$`2`
[1] 0.1833282


$low
$low$`1`
[1] 0.2418546

$low$`2`
[1] 0.1791825

In [16]:
calcErrBarsHerit("50")

$val
$val$`1`
[1] NaN

$val$`2`
[1] NaN


$up
$up$`1`
[1] 0.375259

$up$`2`
[1] 0.344936


$low
$low$`1`
[1] 0.3637281

$low$`2`
[1] 0.3354551

In [19]:
calcErrBarsHerit("30150") 

# this shows a difference --> probably worth looking into

$val
$val$`1`
[1] NaN

$val$`2`
[1] NaN


$up
$up$`1`
[1] 0.177832

$up$`2`
[1] 0.2075617


$low
$low$`1`
[1] 0.1626212

$low$`2`
[1] 0.1959509

In [20]:
calcErrBarsHerit("20022") 


$val
$val$`1`
[1] NaN

$val$`2`
[1] NaN


$up
$up$`1`
[1] 0.1518512

$up$`2`
[1] 0.1014882


$low
$low$`1`
[1] 0.1178345

$low$`2`
[1] 0.08733553

In [22]:
res <- .Last.value


X1,X2,X1.1,X2.1
0.1518512,0.1014882,0.1178345,0.08733553


In [24]:
res.df <- rbind(res$low, res$up)
rownames(res.df) <- c("l", "u")
colnames(res.df) <- c("f", "m")

1,2
0.1178345,0.08733553
0.1518512,0.1014882


Q - is it the case that these differences in variability depend on the x chromosome? If so, which of these are due to X chromosome effects vs other effects?

In [15]:
whr.info <- read.table("../data/dat_set/summary_dat_whr_v2.txt", header=TRUE)
whr.info[,c("h.f", "h.m", "h.auto.f", "h.auto.m")]

# ok so there are different heritabilities if we include auto vs full
# - for this trait they're not hugely different

h.f,h.m,h.auto.f,h.auto.m
0.2693287,0.1958753,0.2697892,0.193606


trait,N,pi.1.,pi.2.,Sigma.1.1.,Sigma.1.2.,Sigma.2.1.,Sigma.2.2.,num1,num2,⋯,rg.l,rg.u,h.f,h.m,h.x.f,h.x.m,h.xy.f,h.xy.m,h.auto.f,h.auto.m
whr,281790,0.7042783,0.2957217,4.044485e-05,2.620484e-05,2.620484e-05,3.008413e-05,278216,3574,⋯,0.7338022,0.7680993,0.2693287,0.1958753,0.2491786,0.3314744,0.3539003,0.2890246,0.2697892,0.193606


In [16]:
res <- list("up"=list("1"=1, "2"=2), "down"=list("1"=3, "2"=4))

In [35]:
require('tidyr')

In [56]:
my.df <- cbind(t(as.data.frame(res)), trait)

my.df2 <- data.frame(cbind(my.df, rownames(my.df)))
colnames(my.df2) <- c("value", "trait", "h.type")

my.df2 <- my.df2 %>% separate( h.type, c("int", "sex"))
rownames(my.df2) <- NULL

value,trait,int,sex
1,whr,up,1
2,whr,up,2
3,whr,down,1
4,whr,down,2
